<center><h1><b>SUPER RESOLUTION</b></h1></center>
<center><h4>Using Generative adversarial network</h4></center>

## Pre Requisites:
<ul style="list-style-type:circle;">
    <li>Python</li>
    <li>Anaconda </li>
</ul>


## Specification:
<pre>
 * GPU:- Nvidia Geforce RTX 2080 Ti
 * Tensorflow 1.14.0 
 * Keras 2.2.4
 * Scipy 1.2.0
 * Data Dimensions:  
         High Resolution: 512*512
         Low Resolution : 256*256
 * Datset: 16000 images of both High and Low resolution images.

</pre>

## Libraries Required:
<ul style="list-style-type:square;">
    <li><b>Keras:</b> Keras is an open-source neural-network library written in Python. It is capable of running on top of TensorFlow, Microsoft Cognitive Toolkit.
    <li><b>Tensorflow:</b> TensorFlow is a free and open-source software library for dataflow and differentiable programming across a range of tasks.
    <li><b>Skimage:</b> Scikit-image is a collection of algorithms for image processing
    <li><b>Numpy:</b> NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
    <li><b>Scipy:</b> SciPy is a free and open-source Python library used for scientific computing and technical computing.
    <li><b>Matplotlib:</b> Matplotlib is a plotting library for the Python programming language and its numerical mathematics extension NumPy.
</ul>


###  <b><h2>Importing Libraries</h2></b>

In [1]:
import os
import time
import keras
import sys
import cv2
import image_slicer

import tensorflow as tf
import skimage.transform
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import matplotlib.pyplot as plt
import numpy as np
import keras.backend as K

from numpy import array
from keras.models import Model
from keras.layers import Dense, add
from keras.layers import Lambda, Input
from keras.layers.core import Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD, Adam, RMSprop
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import load_model, Model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from skimage.transform import rescale, resize
from scipy.misc import imresize
from skimage import img_as_ubyte
from skimage import data, io, filters
from skimage import measure

from numpy import array
from numpy.random import randint
from numpy import expand_dims



ModuleNotFoundError: No module named 'keras'

### <b><h2>Data Loading and Data Preprocessing: </h2></b>
<b>Data Loading is the process that involves taking the data and loading it where the users can access it.</b>
<br>Function load_path is used for getting all the files present in the given directory. It returns the full path to file from the current directory.</br>

In [2]:
def load_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories = directories + load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories
    

<b>Data Loading from the directory.</b>
<br>This function is used to load the files from a specific directory. This uses load_path function to load the path.</br>

In [3]:
def load_data_from_dirs(directory, ext, no_of_images):
    dirs = load_path(directory)
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in os.listdir(d): 
            if f.endswith(ext):
                image = data.imread(os.path.join(d,f))
                if len(image.shape) > 2:
                    files.append(image)
                    file_names.append(os.path.join(d,f))
                count = count + 1
                if count >= no_of_images :
                        return files
    return files     
            

<b>Loading data and resizing the files.</b>
<br>This function is used for loading the images from the directory and resizing the images.</br>

In [4]:
def load_data_from_dirs_resize(directory, ext, size, no_of_images):
    dirs = load_path(directory)
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in os.listdir(d): 
            if f.endswith(ext):
                rimg = resize(data.imread(os.path.join(d,f)), size)
                pltimg.imsave('./mod/' + 'image_%d.jpg' % count, rimg)
                files.append(rimg)
                file_names.append(os.path.join(d,f))
                count = count + 1
                if count >= no_of_images :
                    return files
    return files     


<b>Loading higher resolution images for the model.</b>

In [5]:
def load_test_data_for_model(directory, ext, number_of_images = 100):
    files = load_data_from_dirs(directory, ext, number_of_images)
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
    x_test_hr = preprocess(files, 'hr')

    return x_test_hr

<b>Loading lower resolution images for the model.</b>

In [6]:
def load_test_data_for_model1(directory, ext, number_of_images = 100): #for lr
    files = load_data_from_dirs(directory, ext, number_of_images)
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
    x_test_lr = preprocess1(files, 'lr')
    
    return x_test_lr
    

This function is used to load the test data i.e lower resolution images.

In [7]:
def load_test_data(directory, ext, number_of_images = 100):
    files = load_data_from_dirs(directory, ext, number_of_images)
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()    
    x_test_lr = preprocess1(files, 'lr')
    
    return x_test_lr

## Data Preprocessing
<b>Data processing is, generally, "the collection and manipulation of items of data to produce meaningful information."</b>
<pre>
Test size is 0.2 out of 1.
Function <b>preprocess</b> and <b>preprocess1</b> are used to normalizing HR and LR images to -1 to 1 range. The reason we do both of those things is because in the process of training our network, we're going to be multiplying (weights) and adding to (biases) these initial inputs in order to cause activations that we then backpropogate with the gradients to train the model. We'd like in this process for each feature to have a similar range so that our gradients don't go out of 
control.
(images_hr.astype(np.float32) - 127.5)/127.5 = convert values from [0,255] to  [-1, 1] i.e if the pixel value is 0, then (0-1227.5)/127.5 gives -1. Similarly it converts all the other pixel to the specified range.
</pre>

In [8]:
test_size = 0.2
def preprocess(files,ip=None):
  brk = int((1-test_size)*len(files))
  x_train = files[:brk]
  x_test = files[brk:]
  
  def hr_images_norm(images):
    images_hr = array(images)
    return (images_hr.astype(np.float32) - 127.5)/127.5
  
  x_train_hr = hr_images_norm(x_train)
  x_test_hr = hr_images_norm(x_test)
    
  if(ip == 'hr'):
    return x_test_hr
  
  return x_train_hr,x_test_hr

def preprocess1(files1,ip=None):
  brk = int((1-test_size)*len(files1))
  x_train = files1[:brk]
  x_test = files1[brk:]
  
  def lr_images_norm(images_real):
    images_lr = array(images_real)
    return (images_lr.astype(np.float32) - 127.5)/127.5
  
  x_train_lr = lr_images_norm(x_train)
  x_test_lr = lr_images_norm(x_test)
  
  if(ip == 'lr'):
    return x_test_lr
  
  return x_train_lr,x_test_lr

def denormalize(input_data):
    print('Denormalizing....')
    input_data = ((input_data + 1) * 127.5).astype(np.uint8)
    return input_data

### Dataset Preparattion.
Use this code only for preparing the dataset. Otherwise you can directly use the dataset.

In [9]:
#this code snippet is used to divide the images into 4 exact parts.
input_dir_HR='./testhr' #Please mention the path
input_dir_LR  = './test_LR/' #Please mention the path
image_list_HR = os.listdir(input_dir_HR)
image_list_LR = os.listdir(input_dir_LR)
import image_slicer
for x in range(1379):
    tiles=image_slicer.slice(image_list_HR[x], 4, save=False)
    image_slicer.save_tiles(tiles, directory='./kk', prefix='slice_'+ str(x) , format='jpg')
    
    
#this code snippet is used for scaling the high resolution image to twice that of the higher resolution image.   
from skimage.transform import rescale, resize, downscale_local_mean
input_dir_HR='./TestLR_New'
image_list_HR = os.listdir(input_dir_HR)
image_list_HR.sort()
for x in range(1380):
  image=plt.imread(image_list_HR[x])
  image_rescaled = resize(image, (int(image.shape[0] * 2), int(image.shape[1] * 2)), anti_aliasing=False)
  os.chdir("/gdrive/My Drive/Colab Notebooks/SRGAN-tensorflow/data/LR_img")
  plt.imsave(str(image_list_HR[x]) ,image_rescaled)
  os.chdir("/gdrive/My Drive/Colab Notebooks/SRGAN-tensorflow/data/HR_img")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './testhr'

## Plotting


In [10]:
def plot_generated_images(output_dir, epoch, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5)):
    examples = x_test_hr.shape[0]
    value = randint(0, examples)
    image_batch_hr = denormalize(x_test_hr)
    image_batch_lr = x_test_lr
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    image_batch_lr = denormalize(image_batch_lr)
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[value], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[value], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[value], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir + 'generated_image_%d.png' % epoch)
    
# Plots and save generated images(in form LR, SR, HR) from model to test the model 
# Save output for all images given for testing  
def plot_test_generated_images_for_model(output_dir, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5)):
    examples = x_test_hr.shape[0]
    image_batch_lr = x_test_lr
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    image_batch_lr = denormalize(image_batch_lr)
    image_batch_hr = denormalize(x_test_hr)
    for index in range(examples):
        print("Plotting....")
        plt.figure(figsize=figsize)
        plt.subplot(dim[0], dim[1], 1)
        plt.imshow(image_batch_lr[index], interpolation='nearest')
        d1 = image_batch_lr[index].shape
        plt.axis('off')
        plt.title("Low-Resolution Image : %dx%dx%d"%(d1[0],d1[1],d1[2]))
        plt.subplot(dim[0], dim[1], 2)
        plt.imshow(generated_image[index], interpolation='nearest')
        d2 = generated_image[index].shape
        plt.axis('off')
        plt.title("Super-Resolution Image: %dx%dx%d"%(d2[0],d2[1],d2[2]))
        plt.subplot(dim[0], dim[1], 3)
        plt.imshow(image_batch_hr[index], interpolation='nearest')
        d3 = generated_image[index].shape
        plt.axis('off')
        plt.title("High-Resolution Image: %dx%dx%d"%(d3[0],d3[1],d3[2]))
        plt.tight_layout()
        print("Saving to output_dir....")
        plt.savefig(output_dir + 'test_generated_image_%d.jpg' % index)

# Takes LR images and save respective HR images
def plot_test_generated_images(output_dir, generator, x_test_lr):
    examples = x_test_lr.shape[0]
    image_batch_lr = x_test_lr
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    for index in range(examples):
        pltimg.imsave(output_dir + 'high_res_result_image_%d.jpg' % index, generated_image[index])

###  <center><h2>Network Architecture</h2> </center>
<img src="SRGAN.jpg" alt="SRGAN Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES." width="700" height="700" >



## <center>Residual Block </center>
<img src="resnet.JPG" alt="Resnet Block Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES." align="center">
Shape of our HR image is 512*512. So that we can compare it with the super resolution image. 

In [11]:
image_shape = (512,512,3)
def res_block_gen(model, kernal_size, filters, strides):    
    gen = model
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = add([gen, model])
    return model

### Upsampling Block
<img src="upsampling.JPG" alt="Upsampling Block Diagram. PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="300" height="300">

In [12]:
def up_sampling_block(model, kernal_size, filters, strides):
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = UpSampling2D(size = 2)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    return model

### Discriminator Block
<img src="discriminator1.JPG" alt="Discriminator Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="300" height="300">


In [13]:
def discriminator_block(model, filters, kernel_size, strides):
    model = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    return model

### <center> Network Architecture of Generator</center>
<img src="gen.JPG" alt="Generator Diagram. PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="800" height="800">
<pre>
The generator aims at reproducing sharp images. The network is based on ResNet blocks. The core is 16 ResNet blocks applied to an upsampling of the original image. This ResNet layer is basically a convolutional layer, with input and output added to form the final output.

The final layer outputs a 512x512x3 tensor squashed between values of -1 and 1 through the Hyperbolic Tangent (tanh) function. Finally, we scale the input data to the interval of -1 to 1 to follow the choice of using the tanh function
</pre>

In [14]:
class Generator(object):

    def __init__(self, noise_shape):
        
        self.noise_shape = noise_shape

    def generator(self):
                
        gen_input = Input(shape = self.noise_shape)
        model = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(gen_input)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
        gen_model = model
        
    # Using 16 Residual Blocks
        for index in range(16):
            model = res_block_gen(model, 3, 64, 1)
    
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(model)
        model = BatchNormalization(momentum = 0.5)(model)
        model = add([gen_model, model])
    
    # Using 1 UpSampling Blocks
        for index in range(1):
            model = up_sampling_block(model, 3, 64, 1)
    
        model = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(model)
        model = Activation('tanh')(model)
   
        generator_model = Model(inputs = gen_input, outputs = model)
        #print(generator_model.summary())
        return generator_model

### <center>Network Architecture of Discriminator</center>
<img src="discriminator.JPG" alt="Discriminator Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="800" height="800">
<pre>
The discriminator is also a 7 layer CNN with BN (except its input layer) and leaky RELU activations.Leaky ReLUs are very popular because they help the gradients flow easier through the architecture.

A regular ReLU function works by truncating negative values to 0. This has the effect of blocking the gradients to flow through the network. Instead of the function being zero, leaky RELUs allow a small negative value to pass through. That is, the function computes the greatest value between the features and a small factor.

The discriminator starts by receiving a 512x512x3 image tensor. The discriminator performs a series of strided 1 and 2  convolutions. 

Finally, the discriminator needs to output probabilities. For that, we use the Logistic Sigmoid activation function on the final logits.
</pre>

In [15]:
class Discriminator(object):
    #image_shape = (512,512,3)
    def __init__(self, image_shape):
        
        self.image_shape = image_shape
    
    def discriminator(self):
        
        dis_input = Input(shape = self.image_shape)
        
        model = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = "same")(dis_input)
        model = LeakyReLU(alpha = 0.2)(model)
        
        model = discriminator_block(model, 32, 3, 2)
        model = discriminator_block(model, 64, 3, 1)
        model = discriminator_block(model, 64, 3, 2)
        model = discriminator_block(model, 128, 3, 1)
        model = discriminator_block(model, 128, 3, 2)
        model = discriminator_block(model, 256, 3, 1)
        model = discriminator_block(model, 256, 3, 2)
        
        model = Flatten()(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha = 0.2)(model)
       
        model = Dense(1)(model)
        model = Activation('sigmoid')(model) 
        
        discriminator_model = Model(inputs = dis_input, outputs = model)
        
        return discriminator_model

## Binary Cross Entropy Loss:

<pre>
Formula Derivation:
Loss function for Binary cross Entropy is given by :
<img src="binary_loss.png" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="800" height="800">
where N is the number of training example.
L(yhat,y)=y*log(yhat) + (1-y)*log(1-yhat)  -------------(1)
Where yhat is the Reconstructed Image and y is the Original image
The label for the data coming from real distribution is Y=1 and Yhat =D(x) where x is the probability distribution of random variable X. D(x) is the discriminator output for real data distribution. From (1)
L(D(x),1) = log(D(x))        -------------(2)
And for data coming from generator the label is y=0 and yhat=D(G(z)) where z is the random probability distribution of random variable Z. D(G(z)) is the output of discriminator.  From (1)
L(D(G(z),0) = log(1-D(G(z))  ------------(3)
The objective of the Discriminator is to correctly classify the real and fake data. Hence eqn(1)	and eqn(2) has to be maximized.
<img src="1.JPG" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="400" height="400" >


From the plot max value at D(x)=1 since D(x) lies between 0 and 1
<img src="2.JPG" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="400" height="400">


From the plot max value at D(G(z))=0 since D(G(z)) lies between 0 and 1


Max(log(D(x)) + log(1-D(G(z)))   -----------(4)
The objective of the generator is to fool the discriminator. To achieve this D(G(z)=1.
Therefore we have to minimize eqn(3) and eqn(2) has no role to play since it corresponds to real data.
Min(log(1-D(G(z)))  -------------(5)
<img src="2.JPG" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="400" height="400">

By combining eqn(4) and eqn(5) we can write 
<img src="min_max.JPG" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="800" height="800">

<img src="forward_backward.JPG" alt="Loss formula Diagram.  PLEASE GIVE PROPER PATH TO LOAD IMAGES."  width="800" height="800">
During Forward propagation, noise image is given to generator which produces super resoluted images G(s). This is then given to discriminator along with the real image, which gives the probability of 0 to 1.

During back propagation, after finding the probabilities, we differentiate the cost function and it updates the weights and biases of generator and discriminiator.
</pre>

In [1]:
class BGG_LOSS(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape

    # computes content loss
    def bgg_loss(self, y_true, y_pred):
    
        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=self.image_shape)
        vgg19.trainable = False
        # Make trainable as False
        for l in vgg19.layers:
            l.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False
    
        return K.mean(K.square(model(y_true) - model(y_pred)))

### Combine Generator and Discriminator

In [ ]:
def get_gan_network(discriminator, shape, generator, optimizer):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=[BGG_LOSS(image_shape).bgg_loss, "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)
    return gan

## <center>Training</center>


### Training SRGAN:

In [ ]:
np.random.seed(10)
image_shape = (512,512,3)# for high resolution image
    
def train(files,files1, epochs=100, batch_size=8):
    x_train_hr,x_test_hr = preprocess(files)
    x_train_lr,x_test_lr = preprocess1(files1)
    print("data processed")

    d_loss_real = 0
    d_loss_fake = 0
    loss_gan = 0

    batch_count = int(x_train_hr.shape[0] / batch_size)
    shape = (256, 256, 3) # for low resolution image

    generator = Generator(shape).generator()
    discriminator = Discriminator(image_shape).discriminator()

    adam = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    generator.compile(loss=BGG_LOSS(image_shape).bgg_loss, optimizer=adam)
    discriminator.compile(loss="binary_crossentropy", optimizer=adam)

    shape = (256, 256, 3)
    gan = get_gan_network(discriminator, shape, generator, adam)
    loss_file_name = 'loss_'+time.ctime().replace(' ','_').replace(':','')[11:-5]+'.txt'
    loss_file = open(loss_file_name,'a')
    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batch_count):
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
            generated_images_sr = generator.predict(image_batch_lr)
            print('Batch-%d is fed to generator.'%(_+1))
            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2

            discriminator.trainable = True
            print("Calculating Losses....")
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            print("Training....d_loss_real")
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            print("Training....d_loss_fake")
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
    
            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            loss_gan = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])
            print("Training....loss_gan")

        print("Loss HR , Loss LR, Loss GAN , Dis Loss")
        print(d_loss_real, d_loss_fake, loss_gan , d_loss)
        loss_str = ' '.join(list(map(str,[d_loss_real, d_loss_fake, loss_gan , d_loss])))
        loss_file.write(loss_str+'\n')
        
        if e % (epochs//1) == 0:
            plot_generated_images('./output/', e, generator, x_test_hr, x_test_lr)
        if e % epochs == 0:
            generator.save('./output/gen_model%d.h5' % e)
            discriminator.save('./output/dis_model%d.h5' % e)
            gan.save('./output/gan_model%d.h5' % e)

### Invoking Training function:

In [ ]:
#run this code for training your model
files = load_data_from_dirs("./data1/hr512jpg", ".jpg", no_of_images=100) #loading hr images
files1 = load_data_from_dirs("./data1/lr256jpg", ".jpg", no_of_images=100)#loading lr images
print("data loaded")
train(files,files1,100,2)# train(hr,lr,epoch,batch_size)

## Results After Training:
<pre>
 * Generator Loss vs Epoch: -
<img src="Generator Loss 2.png" alt="Image not loaded. Please check the path.">
 * Image obtained after 1000 epoch:- 
<img src="1000epoochgenerated_image_900.png" alt="Image not loaded. Please check the path."> 
</pre>


# <center>Testing</center>
Low Resolution image has the size 256*256

In [ ]:
#run this code to test your model
image_shape = (256,256,3)

def test_model(input_hig_res,input_low_res, model, number_of_images, output_dir):
    
    x_test_hr = load_test_data_for_model(input_hig_res, 'jpg', number_of_images) 
    x_test_lr = load_test_data_for_model1(input_low_res, 'jpg', number_of_images)
    plot_test_generated_images_for_model(output_dir, model, x_test_hr, x_test_lr)

def test_model_for_lr_images(input_low_res, model, number_of_images, output_dir):

    x_test_lr = load_test_data(input_low_res, 'jpg', number_of_images)
    plot_test_generated_images(output_dir, model, x_test_lr)

if __name__== "__main__":
    #give the path accordingly
    model_dir = './1000epooch/gen_model900.h5'
    test_type = 'test_model'
    input_hig_res = './high_image/'# Please specify the path
    number_of_images = 1 #How many number of images to test the model
    output_dir = './Output/' #Please specify the path
    input_low_res = './Low_image/' Please specify the path
    loss = BGG_LOSS(image_shape)
    model = load_model(model_dir , custom_objects={'bgg_loss': loss.bgg_loss})
    
    if test_type == 'test_model':
        test_model(input_hig_res,input_low_res, model, number_of_images, output_dir)
        
    elif test_type == 'test_lr_images':
        test_model_for_lr_images(input_low_res, model, number_of_images, output_dir)
        
    else:
        print("No such option")

## <center> Performance Metrics</center>
<pre>
              <b><h3> * PSNR</b></h3>
              <b><h3> * SSIM</b></h3>
</pre>
    


### MATLAB code to compute PSNR

In [ ]:
TargetImage=imread(‘Target.jpg’);
SuperResolutedImage=imread(‘super resoluted.jpg’);
n=size(TargetImage);
M=n(1);
N=n(2);
MSE = sum(sum((TargetImage-SuperResolutedImage).^2))/(M*N);
PSNR = 10*log10(256*256/MSE);
fprintf('\nMSE: %7.2f ', MSE);
fprintf('\nPSNR: %9.7f dB', PSNR);

### SSIM:
<b>The Structural Similarity Index (SSIM) is a perceptual metric that quantifies image quality degradation* caused by processing such as data compression or by losses in data transmission.</b>


In [ ]:
 def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    s = measure.compare_ssim(imageA, imageB)

    # setup the figure
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
    
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(image…
           
super_org = cv2.imread('give path here')
original = cv2.imread('give path here')
# convert the images to grayscale
original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
super_org = cv2.cvtColor(super_org, cv2.COLOR_BGR2GRAY)

# initialize the figure
fig = plt.figure("Images")
images = ("Original", original), ("Super", super_org)
 
# loop over the images
for (i, (name, image)) in enumerate(images):
    # show the image
    ax = fig.add_subplot(1, 3, i + 1)
    ax.set_title(name)
    plt.imshow(image)
    plt.axis("off")
 
# show the figure
plt.show()
 
# compare the images
compare_images(original, original, "Original vs. Original")
compare_images(original, super_org, "Original vs. Super")               

<img src="ssmi.jpeg" alt="Image not loaded">


## <center>Feature Maps</center>


In [ ]:
model = load_model('./1000epooch/gen_model900.h5' , custom_objects={'bgg_loss': loss.bgg_loss})
model = Model(inputs=model.inputs, outputs=model.layers[1].output)
# load the imImageage with the required shapelr_trail/bird.jpg
img = load_img('./lr_trail/bird.jpg', target_size=(256, 256))
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)
# get feature map for first hidden layer
feature_maps = model.predict(img)
# plot all 64 maps in an 8x8 squares
square = 8
ix = 1
pyplot.figure(figsize=(15,15))
for _ in range(square):
    for _ in range(square):
        # specify subplot and turn of axis
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale        
        plt.imshow(feature_maps[0, :, :, ix-1])
        ix += 1
    # show the figure
plt.show()
print(i)
plt.imshow(feature_maps[0, :, :, 1])
plt.savefig('./output/layer1' .jpg')

## <center>Input</center>
<img src="bird.jpg" alt="Image not loaded">

## <center>Layer 1</center>

<img src="layer1.jpg" alt="Image not loaded">


## <center>Box Plot</center>
<img src="psnr.png" alt="Image not loaded">
<img src="psnr22.png" alt="Image not loaded" align='left' width='50%' height='50%'>
<img src="ssmi1.png" alt="Image not loaded" align='right' width='50%' height='50%'>

## References:
- Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network by Christian Ledig, Lucas Theis, Ferenc Husz´ar, Jose Caballero, Andrew Cunningham, Alejandro Acosta, Andrew Aitken, Alykhan Tejani, Johannes Totz, Zehan Wang, Wenzhe Shi Twitter
- https://github.com/deepak112/Keras-SRGAN
